# Signing and verifying a block header 

## 1. Creating Encryption Key pair

In [1]:
def generate_RSA(bits=2048):
    '''
    Generate an RSA keypair with an exponent of 65537 in PEM format
    param: bits The key length in bits
    Return private key and public key
    '''
    from Crypto.PublicKey import RSA 
    new_key = RSA.generate(bits, e=65537) 
    public_key = new_key.publickey().exportKey("PEM") 
    private_key = new_key.exportKey("PEM") 
    return private_key, public_key


In [2]:
from Crypto.PublicKey import RSA 

alice_priv, alice_pub = generate_RSA()
bob_priv, bob_pub = generate_RSA()

with open("resources/alice_pub.pem", "wb") as file:
    file.write(alice_pub)

with open("resources/alice_priv.pem", "wb") as file:
    file.write(alice_priv)

with open("resources/bob_pub.pem", "wb") as file:
    file.write(bob_pub)

with open("resources/bob_priv.pem", "wb") as file:
    file.write(bob_priv)

## 2. Signing a blocker header using private key

In [3]:
import json 
from Crypto.Hash import SHA

def hash_block(msg=""):
    # Hashes a list of dictionaries
    if type(msg)!=str:
        msg = json.dumps(msg,sort_keys=True)  # If we don't sort keys, we can't guarantee repeatability!
        
    return SHA.new(str.encode(msg))

In [4]:
# Load resources
with open("resources/alice_priv.pem", "rb") as file:
    alice_priv = RSA.importKey(file.read())
    
with open('resources/alice_pub.pem', "rb") as file:
    alice_pub = RSA.importKey(file.read())

with open("resources/bob_priv.pem", "rb") as file:
    bob_priv = RSA.importKey(file.read())
    
with open('resources/bob_pub.pem', "rb") as file:
    bob_pub = RSA.importKey(file.read())

In [5]:
from Crypto.Signature import PKCS1_v1_5

txns = [
    {"Bob": -50, "Alice": 50},
    {"Bob": -25, "Alice": 25}
]

block_header = {u'blockNumber':0, u'parentHash':None, u'txn':txns}
hash_obj = hash_block(txns)
signer = PKCS1_v1_5.new(bob_priv)
signature = signer.sign(hash_obj)

signature

b"9\x8a(~\xae\x1c\xc8\xc9r\x9bR\x8c\x15\xe1\xd6Y\xe5^\tt\xfd\xcf\xa7\xf1D\xd4\xaeG=\xff0\t\xdb{\x02\xf8\xf1\xc0\x12J\xae\xec\x1c/<t\xe9\x1f\x1eD\x9c#\xcf\x91\x1e\xe7e\x00{T\x18\xa2A\xea\xbc?.\x90\xfc\n\xf3\x08\x7f\xb2\xe4\x1c|U7 \x0f\xb7(N\xb6\x0b\x041\xa3\xbcn\xd5\xa7^u\xc5\x82\x82\xe2\xec\x98\x17\xec\x14\xe8\x98\nK\x93\xe0K\x19.~)\x8agi\xb9\xe9\xdaa\x0c\xdc\xc7\xab\\\xa8\x86\xf1\x96\xb4\xee\xd1\x89h\x062G\xe9<\x12\xa6&+\x9cw\xa4d\xff\xb2\x1f\x93\x85H\x91rfy\xfc\x90\x18\xfe\x9cs\x91\xe5x\xd6\xd3\xf1\xb0!\x89\xe8'<\xec\xaah\xa5|\x9d#\x9d\x18\xaf!r\x85\xcd\x88\xae\x13\x11\xc7T\xd7t/\x8fp/\xb0+\xc9\x08^\xe1\x87\x0e\xe4pE\xbeO\xf1\x0f\xd6\xcaTN^\xdb>\xe0fQ\xf2s\x95\x9d|\xfaC)\xa6C\xfb-\xacL\xb2\xa6a|b\x80#@\x99I\xdf\xed=\x94"

In [6]:
import pickle

with open("resources/signature.pkl", "wb") as file: 
    pickle.dump(signature, file)

## 3. Verifying a block header using public key

In [10]:
from Crypto.Signature import PKCS1_v1_5
import pickle

with open("resources/signature.pkl", "rb") as file: 
    signature = pickle.load(file)

txns = [
    {"Bob": -50, "Alice": 50},
    {"Bob": -25, "Alice": 25}
]

block_header = {u'blockNumber':0, u'parentHash':None, u'txn':txns}
hash_obj = hash_block(txns)

verifier = PKCS1_v1_5.new(bob_pub)

result = verifier.verify(hash_obj, signature)

result

True